<a href="https://colab.research.google.com/github/heinohen/tko_7095_i2hlt/blob/main/week6_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 12: Analogy evaluation

Read in the analogy tuples from the file

In [10]:
!wget https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt

--2024-04-17 10:48:50--  https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.04s   

2024-04-17 10:48:50 (13.1 MB/s) - ‘questions-words.txt’ saved [603955/603955]



In [ ]:
#012
#: capital-common-countries
#Athens Greece Baghdad Iraq
#Athens Greece Bangkok Thailand
#Athens Greece Beijing China
#...

#012
#: capital-world
#Abuja Nigeria Accra Ghana
#Abuja Nigeria Algiers Algeria
#Abuja Nigeria Amman Jordan
#...

#012
#: currency
#Algeria dinar Angola kwanza
#Algeria dinar Argentina peso
#Algeria dinar Armenia dram


In [39]:
## WORKFLOW ##
# 1) create dict to store all 14
# 2) line by line for file
# 3) looking at above see that index 2 start of actual taskname
# 4) format it and create dict entry
# 5) create a list of tuples 'taskitems' to store in dict under key == 'taskname'
# 6) return it
## END ##

# 1)
task_dict = dict() # key == task, value == list of tuples

fname = 'questions-words.txt'
counter = 0

with open(fname) as f:
  # 2)
  for r in f:
    if not r:
      continue # skip empty
    # 3)
    if r.startswith(":"):
      counter += 1
      print(counter) # if we are at start of a task, create entry for dict
      task_name = r[2:].strip('\n')
      print(task_name)
      taskitems = []
      task_dict[task_name] = taskitems

    else:
      #012
      #: capital-common-countries
      #1    " " 2  " "  3  " "  4
      #Athens Greece Baghdad Iraq
      #Athens Greece Bangkok Thailand
      #Athens Greece Beijing China
      #print(r.strip('\n').split(" "))
      taskitems.append(tuple([word for word in r.strip('\n').split(" ")]))





1
capital-common-countries
2
capital-world
3
currency
4
city-in-state
5
family
6
gram1-adjective-to-adverb
7
gram2-opposite
8
gram3-comparative
9
gram4-superlative
10
gram5-present-participle
11
gram6-nationality-adjective
12
gram7-past-tense
13
gram8-plural
14
gram9-plural-verbs


In [37]:

## CHECK ##

print(task_dict['capital-common-countries'][0]) # Athens Greece Baghdad Iraq
print(task_dict['capital-world'][0]) # Abuja Nigeria Accra Ghana
print(task_dict['currency'][0]) # Algeria dinar Angola kwanza

if counter == 14: # It consists of 9+5=14 sets of word analogies.
  print(f'tasks: {counter}')
else:
  print(f'some tasks missing, count is {counter}')
## END ##


('Athens', 'Greece', 'Baghdad', 'Iraq')
('Abuja', 'Nigeria', 'Accra', 'Ghana')
('Algeria', 'dinar', 'Angola', 'kwanza')
tasks: 14
